<a href="https://colab.research.google.com/github/nhuyen183/LungCancerSupportSystem/blob/master/BRFSS_LogisticRegression9JANNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import Libraries { vertical-output: true, display-mode: "both" }

import sqlite3
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

!pip install category_encoders
from category_encoders import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 3.4 MB/s eta 0:00:00


In [2]:
#@title df_structure { display-mode: "both" }

from  glob import glob
filenames = glob('./part*.csv')
appended_data = []  # create a list
for f in filenames:
  df = pd.read_csv(f, index_col=None)
  appended_data.append(df)  # append to the list
df_structure = pd.concat(appended_data, axis=0)
df_structure.info()

ValueError: ignored

In [ ]:
df=df_structure

In [ ]:
#@title HasLungCancer { display-mode: "both" }
print (df["HasLungCancer"].value_counts())

In [ ]:
df.info()

In [ ]:
# Create correlation matrix
correlation = df.select_dtypes("number").drop(columns="HasLungCancer").corr()

# Plot heatmap of `correlation`
sns.heatmap(correlation);

In [ ]:
majority_class_prop, minority_class_prop = y.value_counts(normalize=True)
print(majority_class_prop, minority_class_prop)

In [ ]:
X = df.drop(columns='HasLungCancer').values
y = df['HasLungCancer'].values

from imblearn.over_sampling import SMOTE
# transform the dataset
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


# Initializing the ANN
ann = tf.keras.models.Sequential()

# Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=200, input_dim=13, activation='relu'))
# Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Adding the output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

# Compiling the ANN
ann.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy', 'binary_crossentropy'])
print(ann.summary())

# Training the ANN on the Training set
history = ann.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 100, epochs =10, verbose=1)

In [ ]:
plt.plot(history.history['binary_crossentropy'])
plt.title('Loss Function Over Epochs')
plt.ylabel('loss value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
y_pred = classifier.predict_proba(X_test)

#y_predInverse = sc.inverse_transform(y_pred)
#y_testInverse = sc.inverse_transform(y_test)

In [ ]:
def buildModel(optimizer):
    # Initialising the ANN
    classifier = Sequential()
    
    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 200, input_dim = 13, activation = 'relu'))
    
    # Adding the second hidden layer
    classifier.add(Dense(units = 100, activation = 'relu'))
    
    # Adding the output layer
    classifier.add(Dense(units = 1, activation = 'sigmoid'))
    
    # Compiling the ANN
    classifier.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['mean_absolute_error', 'binary_crossentropy'])
    
    return classifier

In [ ]:
from sklearn.model_selection import GridSearchCV 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

classifier = KerasRegressor(build_fn = buildModel)
#What hyperparameter we want to play with
parameters = {'batch_size': [13, 26, 39, 50],
              'epochs': [40, 50],
              'optimizer': ['adam', 'binary_crossentropy']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'neg_mean_absolute_error',
                           cv = 5)
grid_search = grid_search.fit(X_train, y_train, verbose = 0)

In [ ]:
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: " + str(best_parameters))

In [ ]:
# Build model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

classifier = make_pipeline(MinMaxScaler(), LogisticRegression(max_iter=2000))

# Fit model to training data
history = classifier.fit(X_train, y_train)
print(classifier)

In [ ]:
#@title Evaluate the model { display-mode: "both" }
acc_train = accuracy_score(y_train, classifier.predict(X_train))
acc_test = classifier.score(X_test, y_test)

print("Training Accuracy:", round(acc_train, 4))
print("Test Accuracy:", round(acc_test, 4))

In [ ]:
#acc_baseline = y_train.value_counts(normalize=True).max()
#print("Baseline Accuracy:", round(acc_baseline, 2))

# Predicting the Test set results
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
acc = accuracy_score(y_test, y_pred)
metrics = classification_report(y_test, y_pred)
print(metrics)
print(acc)

In [ ]:
import seaborn as sns

ax = sns.heatmap(cm, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
classifier.named_steps["logisticregression"].intercept_[0]

In [ ]:
classifier.named_steps["logisticregression"].coef_[0]

In [ ]:
features = classifier.named_steps["minmaxscaler"].get_feature_names_out(['Gender','Age65','BMI','GeneralHealth','Smoked100','SmokerStatus','FirstSmokedAge','LastSmokedAge','AvgNumCigADay','HasCTScan','StopSmoking','HasAsthma','HasChronicDisease'])
importances = classifier.named_steps["logisticregression"].coef_[0]
print(features, importances)

In [ ]:
odds_ratios = pd.Series(np.exp(importances),index=features).sort_values()
odds_ratios

In [ ]:
# Horizontal bar chart, five largest coefficients
odds_ratios.plot(kind="barh")
plt.xlabel("Odds Ratio");

In [ ]:
import matplotlib.pyplot as plt
loss_train = history.history['accuracy']

loss_val = history.history['val_accuracy']

epochs = range(1,51)

plt.plot(epochs, loss_train, 'g', label='Training accuracy')

plt.plot(epochs, loss_val, 'b', label='validation accuracy')

plt.title('Training and Validation accuracy')

plt.xlabel('Epochs')

plt.ylabel('Accuracy')

plt.legend()

plt.show()

This chart shows the features that are least likely to be in a building that is severely damaged.

In [ ]:
import matplotlib.pyplot as plt
loss_train = history.history['train_loss']

loss_val = history.history['val_loss']

epochs = range(1,35)

plt.plot(epochs, loss_train, 'g', label='Training loss')

plt.plot(epochs, loss_val, 'b', label='validation loss')

plt.title('Training and Validation loss')

plt.xlabel('Epochs')

plt.ylabel('Loss')

plt.legend()

plt.show()

In [ ]:
def plotGraph(y_test,y_pred,classifier):
    if max(y_test) >= max(y_pred):
        my_range = int(max(y_test))
    else:
        my_range = int(max(y_pred))
    plt.scatter(range(len(y_test)), y_test, color='blue')
    plt.scatter(range(len(y_pred)), y_pred, color='red')
    plt.title(classifier)
    plt.show()
    return

plotGraph(y_test, y_pred, "test")

In [ ]:
import matplotlib.pyplot as plt
loss_train = history.history['loss']

loss_val = history.history['val_loss']

epochs = range(1,51)

plt.plot(epochs, loss_train, 'g', label='Training loss')

plt.plot(epochs, loss_val, 'b', label='validation loss')

plt.title('Training and Validation loss')

plt.xlabel('Epochs')

plt.ylabel('Loss')

plt.legend()

plt.show()

In [ ]:
loss_train = history.history['accuracy']

loss_val = history.history['val_accuracy']

epochs = range(1,51)

plt.plot(epochs, loss_train, 'g', label='Training accuracy')

plt.plot(epochs, loss_val, 'b', label='validation accuracy')

plt.title('Training and Validation accuracy')

plt.xlabel('Epochs')

plt.ylabel('Accuracy')

plt.legend()

plt.show()